<a href="https://colab.research.google.com/github/currencyfxjle/KNAPSACK_GDF_10.0/blob/main/KNAPSACK_GDF_10_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install gspread
!pip install gspread-dataframe
!pip install --upgrade oauth2client
!pip install xlsxwriter

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os

# List files and folders under My Drive
base_path = '/content/drive/My Drive'
for root, dirs, files in os.walk(base_path):
    print(f"Directory: {root}")
    for directory in dirs:
        print(f"  Sub-directory: {directory}")
    for file in files:
        print(f"  File: {file}")

import os

correct_path = '/content/drive/My Drive/Colab Notebooks/GDF - Knapsack/2025 Project Agenda.gsheet'

if os.path.exists(correct_path):
    print("File exists:", correct_path)
else:
    print("File does NOT exist:", correct_path)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 3.5 MB/s eta 0:00:00
Mounted at /content/drive
Directory: /content/drive/My Drive
  Sub-directory: Risk Modeling
  Sub-directory: Colab Notebooks
  File: Financial Concepts   Solar Energy Greenday Finance.gdoc
  File: config.zip
  File: 2024 Project Allocation Results (7).gsheet
  File: 2024 Project Allocation Results (6).gsheet
  File: 2024 Project Allocation Results (5).gsheet
  File: 2024 Project Allocation Results (4).gsheet
  File: 2024 Project Allocation Results (3).gsheet
  File: 2024 Project Allocation Results (2).gsheet
  File: 2024 Project Allocation Results (1).gsheet
  File: 2024 Project Allocation Results.gsheet
  File: Estados de Resultados - Victor Lizarraga.gdoc
  File: Ajustes  knapsack Lalo.gsheet
  File: Backup 2025.gsheet
  File: Ruta calculadora.gdoc
  File: KNAPSACK - GDF - Last Version 2.1.ipynb
  File: Knapsack Project.gdoc
  File: Notes.gdoc
Directory: /content/drive/My Drive/Risk Modeling
  File: Risk 

# **full project allocation first and falls back to chunk allocation when necessary**

In [ ]:
import gspread
from google.colab import auth
from google.auth import default
import pandas as pd

# Authenticate and Access Google Sheet
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Open Google Sheet
title = "2025 Project Agenda"
sheet = gc.open(title)

# Fetch and Clean Supply Data
def fetch_supply():
    ws = sheet.worksheet("Supply")
    data = ws.get_all_values()
    supply_df = pd.DataFrame(data[1:], columns=data[0])

    # Clean and prepare data
    supply_df = supply_df[supply_df['Take into consideration?'].str.lower() == 'y']
    supply_df['Est Cash Amount'] = pd.to_numeric(supply_df['Est Cash Amount'].replace({',': ''}, regex=True))
    supply_df['Poject_date'] = pd.to_datetime(supply_df['Poject_date'], errors='coerce')

    # Convert GDF Profit to numeric
    supply_df['GDF Profit'] = supply_df['GDF Profit'].str.rstrip('%').astype(float) / 100
    supply_df = supply_df.dropna()

    # Sort projects by Est Cash Amount (descending) and date
    supply_df = supply_df.sort_values(by=['Est Cash Amount', 'Poject_date'], ascending=[False, True])

    # Add additional tracking columns
    supply_df['Original Est Cash Amount'] = supply_df['Est Cash Amount']
    supply_df['Fully Allocated'] = False

    # Add category based on Est Cash Amount
    supply_df['Project Category'] = pd.cut(
        supply_df['Est Cash Amount'],
        bins=[0, 350000, 500000, 800000, 1000000, float('inf')],
        labels=['0-350k', '350k-500k', '500k-800k', '800k-1M', '1M+']
    )

    # Include project state for information purposes
    supply_df['Project State'] = supply_df['State']

    return supply_df

# Fetch and Clean Demand Data
def fetch_demand():
    ws = sheet.worksheet("Demand")
    data = ws.get_all_values()
    demand_df = pd.DataFrame(data[1:], columns=data[0])

    # Clean and prepare data
    demand_df['approximated_appetite'] = pd.to_numeric(demand_df['approximated_appetite'].replace({',': ''}, regex=True))
    demand_df['updated pending amount'] = pd.to_numeric(demand_df['pending_amount'].replace({',': ''}, regex=True))
    demand_df['effective_date_purchaser'] = pd.to_datetime(demand_df['effective_date_purchaser'], errors='coerce')
    demand_df['total_allocated'] = 0  # Initialize tracking column
    demand_df = demand_df.dropna()

    # Add a column to track remaining appetite
    demand_df['remaining_appetite'] = demand_df['approximated_appetite']

    # Add category based on approximated appetite
    demand_df['Purchaser Category'] = pd.cut(
        demand_df['approximated_appetite'],
        bins=[0, 350000, 500000, 800000, 1000000, float('inf')],
        labels=['0-350k', '350k-500k', '500k-800k', '800k-1M', '1M+']
    )

    # Add priority for state
    demand_df['State Priority'] = demand_df['State'].apply(lambda x: 1 if x == 'CA' else 2)

    # Sort demand by approximated_appetite (descending) and state priority
    demand_df = demand_df.sort_values(by=['approximated_appetite', 'State Priority'], ascending=[False, True])

    return demand_df

# Match Supply to Demand
def match_supply_to_demand(supply_df, demand_df):
    allocations = []
    total_allocated = 0

    # Group projects by name
    grouped_supply = supply_df.groupby('Proyect Name')

    for idx, purchaser in demand_df.iterrows():
        remaining_pending_amount = purchaser['updated pending amount']
        remaining_appetite = purchaser['remaining_appetite']
        effective_date = purchaser['effective_date_purchaser']

        # Try to allocate full projects first
        for project_name, group in grouped_supply:
            if group['Fully Allocated'].all():
                continue  # Skip fully allocated projects

            project_total_cost = group['Est Cash Amount'].sum()
            project_date = group['Poject_date'].max()  # Use latest date
            project_state = group['Project State'].iloc[0]
            gdf_profit = group['GDF Profit'].mean()
            project_category = group['Project Category'].iloc[0]

            # Check if the full project can be allocated
            if (project_total_cost <= remaining_pending_amount and
                project_total_cost <= remaining_appetite and
                project_date >= effective_date):

                allocations.append({
                    "Purchaser Name": purchaser['purchaser_name'],
                    "Purchaser State": purchaser['State'],
                    "Purchaser Date": effective_date,
                    "Purchaser Category": purchaser['Purchaser Category'],
                    "Project Name": project_name,
                    "Project Date": project_date,
                    "Project Category": project_category,
                    "Project State": project_state,
                    "GDF Profit": gdf_profit,
                    "Allocated Amount": project_total_cost,
                    "Remaining Appetite After": remaining_appetite - project_total_cost,
                    "Updated Pending Amount After": remaining_pending_amount - project_total_cost,
                    "Original Appetite": purchaser['approximated_appetite']
                })

                # Update tracking
                total_allocated += project_total_cost
                demand_df.at[idx, 'total_allocated'] += project_total_cost
                remaining_pending_amount -= project_total_cost
                remaining_appetite -= project_total_cost
                supply_df.loc[group.index, 'Fully Allocated'] = True

                if remaining_pending_amount <= 0 or remaining_appetite <= 0:
                    break

        # If not fully allocated, fall back to chunk allocation
        for _, project in supply_df.iterrows():
            if project['Fully Allocated']:
                continue  # Skip fully allocated chunks

            project_cost = project['Est Cash Amount']
            project_date = project['Poject_date']
            gdf_profit = project['GDF Profit']
            project_category = project['Project Category']
            project_state = project['Project State']

            if (project_cost <= remaining_pending_amount and
                project_cost <= remaining_appetite and
                project_date >= effective_date):

                allocations.append({
                    "Purchaser Name": purchaser['purchaser_name'],
                    "Purchaser State": purchaser['State'],
                    "Purchaser Date": effective_date,
                    "Purchaser Category": purchaser['Purchaser Category'],
                    "Project ID": project['Project ID'],
                    "Project Name": project['Proyect Name'],
                    "Project Date": project_date,
                    "Project Category": project_category,
                    "Project State": project_state,
                    "GDF Profit": gdf_profit,
                    "Allocated Amount": project_cost,
                    "Remaining Appetite After": remaining_appetite - project_cost,
                    "Updated Pending Amount After": remaining_pending_amount - project_cost,
                    "Original Appetite": purchaser['approximated_appetite']
                })

                # Update tracking
                total_allocated += project_cost
                demand_df.at[idx, 'total_allocated'] += project_cost
                remaining_pending_amount -= project_cost
                remaining_appetite -= project_cost
                supply_df.loc[project.name, 'Fully Allocated'] = True

                if remaining_pending_amount <= 0 or remaining_appetite <= 0:
                    break

        demand_df.at[idx, 'updated pending amount'] = remaining_pending_amount
        demand_df.at[idx, 'remaining_appetite'] = remaining_appetite

    return total_allocated, allocations, supply_df, demand_df

# Main Execution
supply_df = fetch_supply()
demand_df = fetch_demand()

# Perform allocation
total_allocated, allocations, remaining_supply, remaining_demand = match_supply_to_demand(supply_df, demand_df)

# Calculate totals
original_total_appetite = demand_df['approximated_appetite'].sum()
total_pending_amount = original_total_appetite - total_allocated
remaining_supply_value = remaining_supply[remaining_supply['Fully Allocated'] == False]['Est Cash Amount'].sum()

# Display results
print("\nResults Summary:")
print(f"Original Total Unallocated Value (Total Appetite): {original_total_appetite}")
print(f"Original Total Project Value (Est Cash Amount): {supply_df['Original Est Cash Amount'].sum()}")
print(f"Total Allocated: {total_allocated}")
print(f"Remaining Supply Value After: {remaining_supply_value}")
print(f"Total Pending Amount: {total_pending_amount}")

# Save allocations and summary
summary_data = pd.DataFrame([{
    "Metric": "Original Total Unallocated Value (Total Appetite)",
    "Value": original_total_appetite
}, {
    "Metric": "Original Total Project Value (Est Cash Amount)",
    "Value": supply_df['Original Est Cash Amount'].sum()
}, {
    "Metric": "Total Allocated",
    "Value": total_allocated
}, {
    "Metric": "Remaining Supply Value After",
    "Value": remaining_supply_value
}, {
    "Metric": "Total Pending Amount",
    "Value": total_pending_amount
}])

# Filter remaining supply and demand
remaining_supply_df = remaining_supply[remaining_supply['Fully Allocated'] == False]
remaining_demand_df = remaining_demand[remaining_demand['remaining_appetite'] > 0]

with pd.ExcelWriter("allocation_results.xlsx", engine="xlsxwriter") as writer:
    pd.DataFrame(allocations).to_excel(writer, sheet_name="Allocations", index=False)
    summary_data.to_excel(writer, sheet_name="Summary", index=False)
    remaining_supply_df.to_excel(writer, sheet_name="Remaining Supply", index=False)
    remaining_demand_df.to_excel(writer, sheet_name="Remaining Demand", index=False)
    print("Results saved to allocation_results.xlsx")

Full projects no partial chunks

full projects no partial allocations, adding the column of project ID to allocations

In [3]:
import gspread
from google.colab import auth
from google.auth import default
import pandas as pd

# Authenticate and Access Google Sheet
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Open Google Sheet
title = "2025 Project Agenda"
sheet = gc.open(title)

# Fetch and Clean Supply Data
def fetch_supply():
    ws = sheet.worksheet("Supply")
    data = ws.get_all_values()
    supply_df = pd.DataFrame(data[1:], columns=data[0])

    # Clean and prepare data
    supply_df = supply_df[supply_df['Take into consideration?'].str.lower() == 'y']
    supply_df['Est Cash Amount'] = pd.to_numeric(supply_df['Est Cash Amount'].replace({',': ''}, regex=True))
    supply_df['Poject_date'] = pd.to_datetime(supply_df['Poject_date'], errors='coerce')

    # Convert GDF Profit to numeric
    supply_df['GDF Profit'] = supply_df['GDF Profit'].str.rstrip('%').astype(float) / 100
    supply_df = supply_df.dropna()

    # Sort projects by Est Cash Amount (descending) and date
    supply_df = supply_df.sort_values(by=['Est Cash Amount', 'Poject_date'], ascending=[False, True])

    # Add additional tracking columns
    supply_df['Original Est Cash Amount'] = supply_df['Est Cash Amount']
    supply_df['Fully Allocated'] = False

    # Add category based on Est Cash Amount
    supply_df['Project Category'] = pd.cut(
        supply_df['Est Cash Amount'],
        bins=[0, 350000, 500000, 800000, 1000000, float('inf')],
        labels=['0-350k', '350k-500k', '500k-800k', '800k-1M', '1M+']
    )

    # Include project state for information purposes
    supply_df['Project State'] = supply_df['State']

    return supply_df

# Fetch and Clean Demand Data
def fetch_demand():
    ws = sheet.worksheet("Demand")
    data = ws.get_all_values()
    demand_df = pd.DataFrame(data[1:], columns=data[0])

    # Clean and prepare data
    demand_df['approximated_appetite'] = pd.to_numeric(demand_df['approximated_appetite'].replace({',': ''}, regex=True))
    demand_df['updated pending amount'] = pd.to_numeric(demand_df['pending_amount'].replace({',': ''}, regex=True))
    demand_df['effective_date_purchaser'] = pd.to_datetime(demand_df['effective_date_purchaser'], errors='coerce')
    demand_df['total_allocated'] = 0  # Initialize tracking column
    demand_df = demand_df.dropna()

    # Add a column to track remaining appetite
    demand_df['remaining_appetite'] = demand_df['approximated_appetite']

    # Add category based on approximated appetite
    demand_df['Purchaser Category'] = pd.cut(
        demand_df['approximated_appetite'],
        bins=[0, 350000, 500000, 800000, 1000000, float('inf')],
        labels=['0-350k', '350k-500k', '500k-800k', '800k-1M', '1M+']
    )

    # Add priority for state
    demand_df['State Priority'] = demand_df['State'].apply(lambda x: 1 if x == 'CA' else 2)

    # Sort demand by approximated_appetite (descending) and state priority
    demand_df = demand_df.sort_values(by=['approximated_appetite', 'State Priority'], ascending=[False, True])

    return demand_df

# Match Supply to Demand
def match_supply_to_demand(supply_df, demand_df):
    allocations = []
    total_allocated = 0

    # Group projects by name
    grouped_supply = supply_df.groupby('Proyect Name')

    for project_name, group in grouped_supply:
        if group['Fully Allocated'].all():
            continue  # Skip fully allocated projects

        project_total_cost = group['Est Cash Amount'].sum()
        project_chunks = group.to_dict('records')  # Convert group to list of chunks

        # Sort purchasers by remaining appetite (descending)
        purchasers = demand_df[demand_df['remaining_appetite'] > 0].sort_values(
            by=['remaining_appetite'], ascending=False
        ).to_dict('records')

        # Check if all chunks can be allocated to purchasers
        can_allocate_project = True
        temp_allocations = []
        temp_demand = demand_df.copy()

        for chunk in project_chunks:
            if chunk['Fully Allocated']:
                continue  # Skip already allocated chunks

            allocated = False
            for purchaser in purchasers:
                if (purchaser['remaining_appetite'] >= chunk['Est Cash Amount'] and
                    purchaser['effective_date_purchaser'] <= chunk['Poject_date']):

                    # Update purchaser's remaining appetite and pending amount
                    purchaser['remaining_appetite'] -= chunk['Est Cash Amount']
                    purchaser['updated pending amount'] -= chunk['Est Cash Amount']

                    # Record the allocation
                    temp_allocations.append({
                        "Purchaser Name": purchaser['purchaser_name'],
                        "Purchaser State": purchaser['State'],
                        "Purchaser Date": purchaser['effective_date_purchaser'],
                        "Purchaser Category": purchaser['Purchaser Category'],
                        "Project ID": chunk['Project ID'],  # Add Project ID
                        "Project Name": project_name,
                        "Project Date": chunk['Poject_date'],
                        "Project Category": chunk['Project Category'],
                        "Project State": chunk['Project State'],
                        "GDF Profit": chunk['GDF Profit'],
                        "Allocated Amount": chunk['Est Cash Amount'],
                        "Remaining Appetite After": purchaser['remaining_appetite'],
                        "Updated Pending Amount After": purchaser['updated pending amount'],
                        "Original Appetite": purchaser['approximated_appetite']
                    })

                    # Update the temporary demand DataFrame
                    temp_demand.loc[temp_demand['purchaser_name'] == purchaser['purchaser_name'], 'remaining_appetite'] = purchaser['remaining_appetite']
                    temp_demand.loc[temp_demand['purchaser_name'] == purchaser['purchaser_name'], 'updated pending amount'] = purchaser['updated pending amount']

                    allocated = True
                    chunk['Fully Allocated'] = True  # Mark chunk as allocated
                    break

            if not allocated:
                can_allocate_project = False
                break  # Skip this project if any chunk cannot be allocated

        # If all chunks can be allocated, finalize the allocations
        if can_allocate_project:
            allocations.extend(temp_allocations)
            total_allocated += project_total_cost
            demand_df = temp_demand  # Update the main demand_df
            supply_df.loc[group.index, 'Fully Allocated'] = True  # Mark project as fully allocated

    return total_allocated, allocations, supply_df, demand_df

# Main Execution
supply_df = fetch_supply()
demand_df = fetch_demand()

# Perform allocation
total_allocated, allocations, remaining_supply, remaining_demand = match_supply_to_demand(supply_df, demand_df)

# Calculate totals
original_total_appetite = demand_df['approximated_appetite'].sum()
total_pending_amount = original_total_appetite - total_allocated
remaining_supply_value = remaining_supply[remaining_supply['Fully Allocated'] == False]['Est Cash Amount'].sum()

# Display results
print("\nResults Summary:")
print(f"Original Total Unallocated Value (Total Appetite): {original_total_appetite}")
print(f"Original Total Project Value (Est Cash Amount): {supply_df['Original Est Cash Amount'].sum()}")
print(f"Total Allocated: {total_allocated}")
print(f"Remaining Supply Value After: {remaining_supply_value}")
print(f"Total Pending Amount: {total_pending_amount}")

# Save allocations and summary
summary_data = pd.DataFrame([{
    "Metric": "Original Total Unallocated Value (Total Appetite)",
    "Value": original_total_appetite
}, {
    "Metric": "Original Total Project Value (Est Cash Amount)",
    "Value": supply_df['Original Est Cash Amount'].sum()
}, {
    "Metric": "Total Allocated",
    "Value": total_allocated
}, {
    "Metric": "Remaining Supply Value After",
    "Value": remaining_supply_value
}, {
    "Metric": "Total Pending Amount",
    "Value": total_pending_amount
}])

# Filter remaining supply and demand
remaining_supply_df = remaining_supply[remaining_supply['Fully Allocated'] == False]
remaining_demand_df = remaining_demand[remaining_demand['remaining_appetite'] > 0]

with pd.ExcelWriter("allocation_results.xlsx", engine="xlsxwriter") as writer:
    pd.DataFrame(allocations).to_excel(writer, sheet_name="Allocations", index=False)
    summary_data.to_excel(writer, sheet_name="Summary", index=False)
    remaining_supply_df.to_excel(writer, sheet_name="Remaining Supply", index=False)
    remaining_demand_df.to_excel(writer, sheet_name="Remaining Demand", index=False)
    print("Results saved to allocation_results.xlsx")


Results Summary:
Original Total Unallocated Value (Total Appetite): 110880429.1
Original Total Project Value (Est Cash Amount): 111770483
Total Allocated: 63727805
Remaining Supply Value After: 48042678
Total Pending Amount: 47152624.099999994
Results saved to allocation_results.xlsx


RUNNING The code on 2025 updated supply and demand

In [7]:
import gspread
from google.colab import auth
from google.auth import default
import pandas as pd

# Authenticate and Access Google Sheet
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Open Google Sheet
title = "2025 Project Agenda"
sheet = gc.open(title)

# Fetch and Clean Supply Data
def fetch_supply():
    ws = sheet.worksheet("Supply")
    data = ws.get_all_values()
    supply_df = pd.DataFrame(data[1:], columns=data[0])

    # Clean and prepare data
    supply_df = supply_df[supply_df['Take into consideration?'].str.lower() == 'y']
    supply_df['Est Cash Amount'] = pd.to_numeric(supply_df['Est Cash Amount'].replace({',': ''}, regex=True))
    supply_df['Poject_date'] = pd.to_datetime(supply_df['Poject_date'], errors='coerce')

    # Convert GDF Profit to numeric
    supply_df['GDF Profit'] = supply_df['GDF Profit'].str.rstrip('%').astype(float) / 100
    supply_df = supply_df.dropna()

    # Sort projects by Est Cash Amount (descending) and date
    supply_df = supply_df.sort_values(by=['Est Cash Amount', 'Poject_date'], ascending=[False, True])

    # Add additional tracking columns
    supply_df['Original Est Cash Amount'] = supply_df['Est Cash Amount']
    supply_df['Fully Allocated'] = False

    # Add category based on Est Cash Amount
    supply_df['Project Category'] = pd.cut(
        supply_df['Est Cash Amount'],
        bins=[0, 350000, 500000, 800000, 1000000, float('inf')],
        labels=['0-350k', '350k-500k', '500k-800k', '800k-1M', '1M+']
    )

    # Include project state for information purposes
    supply_df['Project State'] = supply_df['State']

    return supply_df

# Fetch and Clean Demand Data
def fetch_demand():
    ws = sheet.worksheet("Demand")
    data = ws.get_all_values()
    demand_df = pd.DataFrame(data[1:], columns=data[0])

    # Clean and prepare data
    demand_df['approximated_appetite'] = pd.to_numeric(demand_df['approximated_appetite'].replace({',': ''}, regex=True))
    demand_df['updated pending amount'] = pd.to_numeric(demand_df['pending_amount'].replace({',': ''}, regex=True))
    demand_df['effective_date_purchaser'] = pd.to_datetime(demand_df['effective_date_purchaser'], errors='coerce')
    demand_df['total_allocated'] = 0  # Initialize tracking column
    demand_df = demand_df.dropna()

    # Add a column to track remaining appetite
    demand_df['remaining_appetite'] = demand_df['approximated_appetite']

    # Add category based on approximated appetite
    demand_df['Purchaser Category'] = pd.cut(
        demand_df['approximated_appetite'],
        bins=[0, 350000, 500000, 800000, 1000000, float('inf')],
        labels=['0-350k', '350k-500k', '500k-800k', '800k-1M', '1M+']
    )

    # Add priority for state
    demand_df['State Priority'] = demand_df['State'].apply(lambda x: 1 if x == 'CA' else 2)

    # Sort demand by approximated_appetite (descending) and state priority
    demand_df = demand_df.sort_values(by=['approximated_appetite', 'State Priority'], ascending=[False, True])

    return demand_df

# Match Supply to Demand
def match_supply_to_demand(supply_df, demand_df):
    allocations = []
    total_allocated = 0

    # Group projects by name
    grouped_supply = supply_df.groupby('Proyect Name')

    for project_name, group in grouped_supply:
        if group['Fully Allocated'].all():
            continue  # Skip fully allocated projects

        project_total_cost = group['Est Cash Amount'].sum()
        project_chunks = group.to_dict('records')  # Convert group to list of chunks

        # Sort purchasers by remaining appetite (descending)
        purchasers = demand_df[demand_df['remaining_appetite'] > 0].sort_values(
            by=['remaining_appetite'], ascending=False
        ).to_dict('records')

        # Check if all chunks can be allocated to purchasers
        can_allocate_project = True
        temp_allocations = []
        temp_demand = demand_df.copy()

        for chunk in project_chunks:
            if chunk['Fully Allocated']:
                continue  # Skip already allocated chunks

            allocated = False
            for purchaser in purchasers:
                if (purchaser['remaining_appetite'] >= chunk['Est Cash Amount'] and
                    purchaser['effective_date_purchaser'] <= chunk['Poject_date']):

                    # Update purchaser's remaining appetite and pending amount
                    purchaser['remaining_appetite'] -= chunk['Est Cash Amount']
                    purchaser['updated pending amount'] -= chunk['Est Cash Amount']

                    # Record the allocation
                    temp_allocations.append({
                        "Purchaser Name": purchaser['purchaser_name'],
                        "Purchaser State": purchaser['State'],
                        "Purchaser Date": purchaser['effective_date_purchaser'],
                        "Purchaser Category": purchaser['Purchaser Category'],
                        "Project ID": chunk['Project ID'],  # Add Project ID
                        "Project Name": project_name,
                        "Project Date": chunk['Poject_date'],
                        "Project Category": chunk['Project Category'],
                        "Project State": chunk['Project State'],
                        "GDF Profit": chunk['GDF Profit'],
                        "Allocated Amount": chunk['Est Cash Amount'],
                        "Remaining Appetite After": purchaser['remaining_appetite'],
                        "Updated Pending Amount After": purchaser['updated pending amount'],
                        "Original Appetite": purchaser['approximated_appetite']
                    })

                    # Update the temporary demand DataFrame
                    temp_demand.loc[temp_demand['purchaser_name'] == purchaser['purchaser_name'], 'remaining_appetite'] = purchaser['remaining_appetite']
                    temp_demand.loc[temp_demand['purchaser_name'] == purchaser['purchaser_name'], 'updated pending amount'] = purchaser['updated pending amount']

                    allocated = True
                    chunk['Fully Allocated'] = True  # Mark chunk as allocated
                    break

            if not allocated:
                can_allocate_project = False
                break  # Skip this project if any chunk cannot be allocated

        # If all chunks can be allocated, finalize the allocations
        if can_allocate_project:
            allocations.extend(temp_allocations)
            total_allocated += project_total_cost
            demand_df = temp_demand  # Update the main demand_df
            supply_df.loc[group.index, 'Fully Allocated'] = True  # Mark project as fully allocated

    return total_allocated, allocations, supply_df, demand_df

# Main Execution
supply_df = fetch_supply()
demand_df = fetch_demand()

# Perform allocation
total_allocated, allocations, remaining_supply, remaining_demand = match_supply_to_demand(supply_df, demand_df)

# Calculate totals
original_total_appetite = demand_df['approximated_appetite'].sum()
total_pending_amount = original_total_appetite - total_allocated
remaining_supply_value = remaining_supply[remaining_supply['Fully Allocated'] == False]['Est Cash Amount'].sum()

# Display results
print("\nResults Summary:")
print(f"Original Total Unallocated Value (Total Appetite): {original_total_appetite}")
print(f"Original Total Project Value (Est Cash Amount): {supply_df['Original Est Cash Amount'].sum()}")
print(f"Total Allocated: {total_allocated}")
print(f"Remaining Supply Value After: {remaining_supply_value}")
print(f"Total Pending Amount: {total_pending_amount}")

# Save allocations and summary
summary_data = pd.DataFrame([{
    "Metric": "Original Total Unallocated Value (Total Appetite)",
    "Value": original_total_appetite
}, {
    "Metric": "Original Total Project Value (Est Cash Amount)",
    "Value": supply_df['Original Est Cash Amount'].sum()
}, {
    "Metric": "Total Allocated",
    "Value": total_allocated
}, {
    "Metric": "Remaining Supply Value After",
    "Value": remaining_supply_value
}, {
    "Metric": "Total Pending Amount",
    "Value": total_pending_amount
}])

# Filter remaining supply and demand
remaining_supply_df = remaining_supply[remaining_supply['Fully Allocated'] == False]
remaining_demand_df = remaining_demand[remaining_demand['remaining_appetite'] > 0]

with pd.ExcelWriter("allocation_results.xlsx", engine="xlsxwriter") as writer:
    pd.DataFrame(allocations).to_excel(writer, sheet_name="Allocations", index=False)
    summary_data.to_excel(writer, sheet_name="Summary", index=False)
    remaining_supply_df.to_excel(writer, sheet_name="Remaining Supply", index=False)
    remaining_demand_df.to_excel(writer, sheet_name="Remaining Demand", index=False)
    print("Results saved to allocation_results.xlsx")


Results Summary:
Original Total Unallocated Value (Total Appetite): 3920000.0
Original Total Project Value (Est Cash Amount): 18020499.4
Total Allocated: 0
Remaining Supply Value After: 18020499.4
Total Pending Amount: 3920000.0
Results saved to allocation_results.xlsx


allowing partialñ chunks allocations per project

In [8]:
import gspread
from google.colab import auth
from google.auth import default
import pandas as pd

# Authenticate and Access Google Sheet
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Open Google Sheet
title = "2025 Project Agenda"
sheet = gc.open(title)

# Fetch and Clean Supply Data
def fetch_supply():
    ws = sheet.worksheet("Supply")
    data = ws.get_all_values()
    supply_df = pd.DataFrame(data[1:], columns=data[0])

    # Clean and prepare data
    supply_df = supply_df[supply_df['Take into consideration?'].str.lower() == 'y']
    supply_df['Est Cash Amount'] = pd.to_numeric(supply_df['Est Cash Amount'].replace({',': ''}, regex=True))
    supply_df['Poject_date'] = pd.to_datetime(supply_df['Poject_date'], errors='coerce')

    # Convert GDF Profit to numeric
    supply_df['GDF Profit'] = supply_df['GDF Profit'].str.rstrip('%').astype(float) / 100
    supply_df = supply_df.dropna()

    # Sort projects by Est Cash Amount (descending) and date
    supply_df = supply_df.sort_values(by=['Est Cash Amount', 'Poject_date'], ascending=[False, True])

    # Add additional tracking columns
    supply_df['Original Est Cash Amount'] = supply_df['Est Cash Amount']
    supply_df['Fully Allocated'] = False

    # Add category based on Est Cash Amount
    supply_df['Project Category'] = pd.cut(
        supply_df['Est Cash Amount'],
        bins=[0, 350000, 500000, 800000, 1000000, float('inf')],
        labels=['0-350k', '350k-500k', '500k-800k', '800k-1M', '1M+']
    )

    # Include project state for information purposes
    supply_df['Project State'] = supply_df['State']

    return supply_df

# Fetch and Clean Demand Data
def fetch_demand():
    ws = sheet.worksheet("Demand")
    data = ws.get_all_values()
    demand_df = pd.DataFrame(data[1:], columns=data[0])

    # Clean and prepare data
    demand_df['approximated_appetite'] = pd.to_numeric(demand_df['approximated_appetite'].replace({',': ''}, regex=True))
    demand_df['updated pending amount'] = pd.to_numeric(demand_df['pending_amount'].replace({',': ''}, regex=True))
    demand_df['effective_date_purchaser'] = pd.to_datetime(demand_df['effective_date_purchaser'], errors='coerce')
    demand_df['total_allocated'] = 0  # Initialize tracking column
    demand_df = demand_df.dropna()

    # Add a column to track remaining appetite
    demand_df['remaining_appetite'] = demand_df['approximated_appetite']

    # Add category based on approximated appetite
    demand_df['Purchaser Category'] = pd.cut(
        demand_df['approximated_appetite'],
        bins=[0, 350000, 500000, 800000, 1000000, float('inf')],
        labels=['0-350k', '350k-500k', '500k-800k', '800k-1M', '1M+']
    )

    # Add priority for state
    demand_df['State Priority'] = demand_df['State'].apply(lambda x: 1 if x == 'CA' else 2)

    # Sort demand by approximated_appetite (descending) and state priority
    demand_df = demand_df.sort_values(by=['approximated_appetite', 'State Priority'], ascending=[False, True])

    return demand_df

# Match Supply to Demand
def match_supply_to_demand(supply_df, demand_df):
    allocations = []
    total_allocated = 0

    # Group projects by name
    grouped_supply = supply_df.groupby('Proyect Name')

    for idx, purchaser in demand_df.iterrows():
        remaining_pending_amount = purchaser['updated pending amount']
        remaining_appetite = purchaser['remaining_appetite']
        effective_date = purchaser['effective_date_purchaser']

        # Try to allocate full projects first
        for project_name, group in grouped_supply:
            if group['Fully Allocated'].all():
                continue  # Skip fully allocated projects

            project_total_cost = group['Est Cash Amount'].sum()
            project_date = group['Poject_date'].max()  # Use latest date
            project_state = group['Project State'].iloc[0]
            gdf_profit = group['GDF Profit'].mean()
            project_category = group['Project Category'].iloc[0]

            # Check if the full project can be allocated
            if (project_total_cost <= remaining_pending_amount and
                project_total_cost <= remaining_appetite and
                project_date >= effective_date):

                allocations.append({
                    "Purchaser Name": purchaser['purchaser_name'],
                    "Purchaser State": purchaser['State'],
                    "Purchaser Date": effective_date,
                    "Purchaser Category": purchaser['Purchaser Category'],
                    "Project Name": project_name,
                    "Project Date": project_date,
                    "Project Category": project_category,
                    "Project State": project_state,
                    "GDF Profit": gdf_profit,
                    "Allocated Amount": project_total_cost,
                    "Remaining Appetite After": remaining_appetite - project_total_cost,
                    "Updated Pending Amount After": remaining_pending_amount - project_total_cost,
                    "Original Appetite": purchaser['approximated_appetite']
                })

                # Update tracking
                total_allocated += project_total_cost
                demand_df.at[idx, 'total_allocated'] += project_total_cost
                remaining_pending_amount -= project_total_cost
                remaining_appetite -= project_total_cost
                supply_df.loc[group.index, 'Fully Allocated'] = True

                if remaining_pending_amount <= 0 or remaining_appetite <= 0:
                    break

        # If not fully allocated, fall back to chunk allocation
        for _, project in supply_df.iterrows():
            if project['Fully Allocated']:
                continue  # Skip fully allocated chunks

            project_cost = project['Est Cash Amount']
            project_date = project['Poject_date']
            gdf_profit = project['GDF Profit']
            project_category = project['Project Category']
            project_state = project['Project State']

            if (project_cost <= remaining_pending_amount and
                project_cost <= remaining_appetite and
                project_date >= effective_date):

                allocations.append({
                    "Purchaser Name": purchaser['purchaser_name'],
                    "Purchaser State": purchaser['State'],
                    "Purchaser Date": effective_date,
                    "Purchaser Category": purchaser['Purchaser Category'],
                    "Project ID": project['Project ID'],
                    "Project Name": project['Proyect Name'],
                    "Project Date": project_date,
                    "Project Category": project_category,
                    "Project State": project_state,
                    "GDF Profit": gdf_profit,
                    "Allocated Amount": project_cost,
                    "Remaining Appetite After": remaining_appetite - project_cost,
                    "Updated Pending Amount After": remaining_pending_amount - project_cost,
                    "Original Appetite": purchaser['approximated_appetite']
                })

                # Update tracking
                total_allocated += project_cost
                demand_df.at[idx, 'total_allocated'] += project_cost
                remaining_pending_amount -= project_cost
                remaining_appetite -= project_cost
                supply_df.loc[project.name, 'Fully Allocated'] = True

                if remaining_pending_amount <= 0 or remaining_appetite <= 0:
                    break

        demand_df.at[idx, 'updated pending amount'] = remaining_pending_amount
        demand_df.at[idx, 'remaining_appetite'] = remaining_appetite

    return total_allocated, allocations, supply_df, demand_df

# Main Execution
supply_df = fetch_supply()
demand_df = fetch_demand()

# Perform allocation
total_allocated, allocations, remaining_supply, remaining_demand = match_supply_to_demand(supply_df, demand_df)

# Calculate totals
original_total_appetite = demand_df['approximated_appetite'].sum()
total_pending_amount = original_total_appetite - total_allocated
remaining_supply_value = remaining_supply[remaining_supply['Fully Allocated'] == False]['Est Cash Amount'].sum()

# Display results
print("\nResults Summary:")
print(f"Original Total Unallocated Value (Total Appetite): {original_total_appetite}")
print(f"Original Total Project Value (Est Cash Amount): {supply_df['Original Est Cash Amount'].sum()}")
print(f"Total Allocated: {total_allocated}")
print(f"Remaining Supply Value After: {remaining_supply_value}")
print(f"Total Pending Amount: {total_pending_amount}")

# Save allocations and summary
summary_data = pd.DataFrame([{
    "Metric": "Original Total Unallocated Value (Total Appetite)",
    "Value": original_total_appetite
}, {
    "Metric": "Original Total Project Value (Est Cash Amount)",
    "Value": supply_df['Original Est Cash Amount'].sum()
}, {
    "Metric": "Total Allocated",
    "Value": total_allocated
}, {
    "Metric": "Remaining Supply Value After",
    "Value": remaining_supply_value
}, {
    "Metric": "Total Pending Amount",
    "Value": total_pending_amount
}])

# Filter remaining supply and demand
remaining_supply_df = remaining_supply[remaining_supply['Fully Allocated'] == False]
remaining_demand_df = remaining_demand[remaining_demand['remaining_appetite'] > 0]

with pd.ExcelWriter("allocation_results.xlsx", engine="xlsxwriter") as writer:
    pd.DataFrame(allocations).to_excel(writer, sheet_name="Allocations", index=False)
    summary_data.to_excel(writer, sheet_name="Summary", index=False)
    remaining_supply_df.to_excel(writer, sheet_name="Remaining Supply", index=False)
    remaining_demand_df.to_excel(writer, sheet_name="Remaining Demand", index=False)
    print("Results saved to allocation_results.xlsx")


Results Summary:
Original Total Unallocated Value (Total Appetite): 3920000.0
Original Total Project Value (Est Cash Amount): 18020499.4
Total Allocated: 3781546.0
Remaining Supply Value After: 14238953.399999997
Total Pending Amount: 138454.0
Results saved to allocation_results.xlsx


Supply & demand summary is completed

In [15]:
import gspread
from google.colab import auth
from google.auth import default
import pandas as pd

# Authenticate and Access Google Sheet
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Open Google Sheet
title = "2025 Project Agenda"
sheet = gc.open(title)

# Fetch and Clean Supply Data
def fetch_supply():
    ws = sheet.worksheet("Supply")
    data = ws.get_all_values()
    supply_df = pd.DataFrame(data[1:], columns=data[0])

    # Clean and prepare data
    supply_df = supply_df[supply_df['Take into consideration?'].str.lower() == 'y']
    supply_df['Est Cash Amount'] = pd.to_numeric(supply_df['Est Cash Amount'].replace({',': ''}, regex=True))
    supply_df['Poject_date'] = pd.to_datetime(supply_df['Poject_date'], errors='coerce')

    # Convert GDF Profit to numeric
    supply_df['GDF Profit'] = supply_df['GDF Profit'].str.rstrip('%').astype(float) / 100
    supply_df = supply_df.dropna()

    # Sort projects by Est Cash Amount (descending) and date
    supply_df = supply_df.sort_values(by=['Est Cash Amount', 'Poject_date'], ascending=[False, True])

    # Add additional tracking columns
    supply_df['Original Est Cash Amount'] = supply_df['Est Cash Amount']
    supply_df['Fully Allocated'] = False

    # Add category based on Est Cash Amount
    supply_df['Project Category'] = pd.cut(
        supply_df['Est Cash Amount'],
        bins=[0, 350000, 500000, 800000, 1000000, float('inf')],
        labels=['0-350k', '350k-500k', '500k-800k', '800k-1M', '1M+']
    )

    # Include project state for information purposes
    supply_df['Project State'] = supply_df['State']

    return supply_df

# Fetch and Clean Demand Data
def fetch_demand():
    ws = sheet.worksheet("Demand")
    data = ws.get_all_values()
    demand_df = pd.DataFrame(data[1:], columns=data[0])

    # Clean and prepare data
    demand_df['approximated_appetite'] = pd.to_numeric(demand_df['approximated_appetite'].replace({',': ''}, regex=True))
    demand_df['updated pending amount'] = pd.to_numeric(demand_df['pending_amount'].replace({',': ''}, regex=True))
    demand_df['effective_date_purchaser'] = pd.to_datetime(demand_df['effective_date_purchaser'], errors='coerce')
    demand_df = demand_df.dropna()

    # Add a column to track remaining appetite
    demand_df['remaining_appetite'] = demand_df['approximated_appetite']

    # Add category based on approximated appetite
    demand_df['Purchaser Category'] = pd.cut(
        demand_df['approximated_appetite'],
        bins=[0, 350000, 500000, 800000, 1000000, float('inf')],
        labels=['0-350k', '350k-500k', '500k-800k', '800k-1M', '1M+']
    )

    # Add priority for state
    demand_df['State Priority'] = demand_df['State'].apply(lambda x: 1 if x == 'CA' else 2)

    # Sort demand by approximated_appetite (descending) and state priority
    demand_df = demand_df.sort_values(by=['approximated_appetite', 'State Priority'], ascending=[False, True])

    return demand_df

# Match Supply to Demand
def match_supply_to_demand(supply_df, demand_df):
    allocations = []
    total_allocated = 0

    # Group projects by name
    grouped_supply = supply_df.groupby('Proyect Name')

    for project_name, group in grouped_supply:
        if group['Fully Allocated'].all():
            continue  # Skip fully allocated projects

        project_total_cost = group['Est Cash Amount'].sum()
        project_chunks = group.to_dict('records')  # Convert group to list of chunks

        # Sort purchasers by remaining appetite (descending)
        purchasers = demand_df[demand_df['remaining_appetite'] > 0].sort_values(
            by=['remaining_appetite'], ascending=False
        ).to_dict('records')

        # Check if all chunks can be allocated to purchasers
        can_allocate_project = True
        temp_allocations = []
        temp_demand = demand_df.copy()

        for chunk in project_chunks:
            if chunk['Fully Allocated']:
                continue  # Skip already allocated chunks

            allocated = False
            for purchaser in purchasers:
                if (purchaser['remaining_appetite'] >= chunk['Est Cash Amount'] and
                    purchaser['effective_date_purchaser'] <= chunk['Poject_date']):

                    # Update purchaser's remaining appetite and pending amount
                    purchaser['remaining_appetite'] -= chunk['Est Cash Amount']
                    purchaser['updated pending amount'] -= chunk['Est Cash Amount']

                    # Record the allocation
                    temp_allocations.append({
                        "Purchaser Name": purchaser['purchaser_name'],
                        "Purchaser State": purchaser['State'],
                        "Purchaser Date": purchaser['effective_date_purchaser'],
                        "Purchaser Category": purchaser['Purchaser Category'],
                        "Project ID": chunk['Project ID'],  # Add Project ID
                        "Project Name": project_name,
                        "Project Date": chunk['Poject_date'],
                        "Project Category": chunk['Project Category'],
                        "Project State": chunk['Project State'],
                        "GDF Profit": chunk['GDF Profit'],
                        "Allocated Amount": chunk['Est Cash Amount'],
                        "Remaining Appetite After": purchaser['remaining_appetite'],
                        "Updated Pending Amount After": purchaser['updated pending amount'],
                        "Original Appetite": purchaser['approximated_appetite']
                    })

                    # Update the temporary demand DataFrame
                    temp_demand.loc[temp_demand['purchaser_name'] == purchaser['purchaser_name'], 'remaining_appetite'] = purchaser['remaining_appetite']
                    temp_demand.loc[temp_demand['purchaser_name'] == purchaser['purchaser_name'], 'updated pending amount'] = purchaser['updated pending amount']

                    allocated = True
                    chunk['Fully Allocated'] = True  # Mark chunk as allocated
                    break

            if not allocated:
                can_allocate_project = False
                break  # Skip this project if any chunk cannot be allocated

        # If all chunks can be allocated, finalize the allocations
        if can_allocate_project:
            allocations.extend(temp_allocations)
            total_allocated += project_total_cost
            demand_df = temp_demand  # Update the main demand_df
            supply_df.loc[group.index, 'Fully Allocated'] = True  # Mark project as fully allocated

    return total_allocated, allocations, supply_df, demand_df

# Main Execution
supply_df = fetch_supply()
demand_df = fetch_demand()

# Perform allocation
total_allocated, allocations, remaining_supply, remaining_demand = match_supply_to_demand(supply_df, demand_df)

# Calculate totals
original_total_appetite = demand_df['approximated_appetite'].sum()
total_pending_amount = original_total_appetite - total_allocated
remaining_supply_value = remaining_supply[remaining_supply['Fully Allocated'] == False]['Est Cash Amount'].sum()

# Display results
print("\nResults Summary:")
print(f"Original Total Unallocated Value (Total Appetite): {original_total_appetite}")
print(f"Original Total Project Value (Est Cash Amount): {supply_df['Original Est Cash Amount'].sum()}")
print(f"Total Allocated: {total_allocated}")
print(f"Remaining Supply Value After: {remaining_supply_value}")
print(f"Total Pending Amount: {total_pending_amount}")

# Create Supply Summary
supply_summary = supply_df.groupby('Proyect Name').agg(
    Total_Chunks_Before=('Est Cash Amount', 'count'),
    Total_Chunks_After=('Fully Allocated', lambda x: x.sum()),
    Total_Project_Value=('Est Cash Amount', 'sum')
).reset_index()

# Create Demand Summary
allocations_df = pd.DataFrame(allocations)

# Group allocations by purchaser
allocations_grouped = allocations_df.groupby('Purchaser Name').agg(
    Original_Appetite=('Original Appetite', 'first'),
    Total_Allocated=('Allocated Amount', 'sum'),
    Remaining_Appetite=('Remaining Appetite After', 'last'),
    Updated_Pending_Amount=('Updated Pending Amount After', 'last')
).reset_index()

# Merge with the original demand data to include purchasers with no allocations
demand_summary = demand_df[['purchaser_name', 'approximated_appetite', 'pending_amount']].rename(
    columns={'purchaser_name': 'Purchaser Name', 'approximated_appetite': 'Original_Appetite', 'pending_amount': 'Updated_Pending_Amount'}
)

# Left join with allocations_grouped to include all purchasers
demand_summary = demand_summary.merge(
    allocations_grouped[['Purchaser Name', 'Total_Allocated', 'Remaining_Appetite']],
    on='Purchaser Name',
    how='left'
)

# Fill NaN values for purchasers with no allocations
demand_summary['Total_Allocated'] = demand_summary['Total_Allocated'].fillna(0)
demand_summary['Remaining_Appetite'] = demand_summary['Remaining_Appetite'].fillna(demand_summary['Original_Appetite'])

# Calculate percentages
demand_summary['Percentage_Allocated'] = (demand_summary['Total_Allocated'] / demand_summary['Original_Appetite']) * 100
demand_summary['Percentage_Remaining'] = (demand_summary['Remaining_Appetite'] / demand_summary['Original_Appetite']) * 100

# Save allocations and summary
summary_data = pd.DataFrame([{
    "Metric": "Original Total Unallocated Value (Total Appetite)",
    "Value": original_total_appetite
}, {
    "Metric": "Original Total Project Value (Est Cash Amount)",
    "Value": supply_df['Original Est Cash Amount'].sum()
}, {
    "Metric": "Total Allocated",
    "Value": total_allocated
}, {
    "Metric": "Remaining Supply Value After",
    "Value": remaining_supply_value
}, {
    "Metric": "Total Pending Amount",
    "Value": total_pending_amount
}])

# Filter remaining supply and demand
remaining_supply_df = remaining_supply[remaining_supply['Fully Allocated'] == False]
remaining_demand_df = remaining_demand[remaining_demand['remaining_appetite'] > 0]

with pd.ExcelWriter("allocation_results.xlsx", engine="xlsxwriter") as writer:
    pd.DataFrame(allocations).to_excel(writer, sheet_name="Allocations", index=False)
    summary_data.to_excel(writer, sheet_name="Summary", index=False)
    remaining_supply_df.to_excel(writer, sheet_name="Remaining Supply", index=False)
    remaining_demand_df.to_excel(writer, sheet_name="Remaining Demand", index=False)
    supply_summary.to_excel(writer, sheet_name="Supply Summary", index=False)
    demand_summary.to_excel(writer, sheet_name="Demand Summary", index=False)
    print("Results saved to allocation_results.xlsx")


Results Summary:
Original Total Unallocated Value (Total Appetite): 110880429.1
Original Total Project Value (Est Cash Amount): 111770483
Total Allocated: 63727805
Remaining Supply Value After: 48042678
Total Pending Amount: 47152624.099999994
Results saved to allocation_results.xlsx


Charts located in code KNAPSACK_GDF_9.0